In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np 
from IPython.display import clear_output 
from six.moves import urllib
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc



In [15]:
dftrain = pd.read_csv('G:/projects/tensorflow/data/train.csv') #importing training data
dfeval = pd.read_csv('G:/projects/tensorflow/data/eval.csv')   #importing testing data
# print(dftrain.head())    #prints the first 5 elements of the dftrain dataset
# print(dfeval.head())     #prints the first 5 values of the dfeval dataset
y_train = dftrain.pop('survived')     #this statemnet pops the survived column from dftrain dataset and store it in y_train variable
y_eval = dfeval.pop('survived')       #this statemnet pops the survived column from dfeval dataset and store it in y_eval variable